In [ ]:
#%conda install nc-time-axis
#%conda install bottleneck
#%conda install netcdftime

In [1]:
#!/usr/bin/env python
# coding: utf-8
import netCDF4 as nc 
import numpy as np
import pandas as pd
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import nc_time_axis
import cftime
from dask.diagnostics import ProgressBar
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import warnings
warnings.simplefilter('ignore')
import datetime
import csv
import os 
import bottleneck as bn
import sys
import intake

In [2]:
def is_jj(month):
    return (month >= 6) & (month <= 7)

def bn_mov_mean(data,width):
    return bn.move_mean(a=data,window=width,min_count = 1)

In [3]:
BASE = "http://esgf-data.ucar.edu/thredds/dodsC/esg_dataroot/CMIP6/"
EXP = "ScenarioMIP/NCAR/CESM2/ssp585/r1i1p1f1/Omon/"
VAR = ["thetao" , "so" , "zos"]
GRID = "/gr/v20190730/"
#SUFFIX = ["_Omon_CESM2_ssp585_r1i1p1f1_gr_201501-206412.nc",\
#           "_Omon_CESM2_ssp585_r1i1p1f1_gr_206501-210012.nc"]
#for j in SUFFIX:

SUFFIX1 = "_Omon_CESM2_ssp585_r1i1p1f1_gr_201501-206412.nc"
SUFFIX2 = "_Omon_CESM2_ssp585_r1i1p1f1_gr_206501-210012.nc"
          
ds_surf1 = xr.Dataset()
for i in VAR:    
    url = BASE + EXP + i + GRID + i + SUFFIX1
    ds = xr.open_dataset(url)
    lat = ds['lat']
    lon = ds['lon']
    time = ds['time']
    
    try:
        surf = ds[i].sel(lev = 0, lat=slice(26,28), lon=slice(267,274))
        surf = surf.sel(time=is_jj(surf['time.month'])).mean(dim = ('lon','lat')).drop('lev')          
        surf.name = ['surf_' + i]
        ds_surf1['surf_' + i] = surf
    except:
        surf = ds[i].sel(lat=slice(26,28), lon=slice(267,274))
        surf = surf.sel(time=is_jj(surf['time.month'])).mean(dim = ('lon','lat'))
        surf.name = ['surf_' + i]
        ds_surf1['surf_' + i] = surf

ds_surf2 = xr.Dataset()
for i in VAR:    
    url = BASE + EXP + i + GRID + i + SUFFIX2
    ds = xr.open_dataset(url)
    lat = ds['lat']
    lon = ds['lon']
    time = ds['time']
    
    try:
        surf = ds[i].sel(lev = 0, lat=slice(26,28), lon=slice(267,274))
        surf = surf.sel(time=is_jj(surf['time.month'])).mean(dim = ('lon','lat')).drop('lev')          
        surf.name = ['surf_' + i]
        ds_surf2['surf_' + i] = surf
    except:
        surf = ds[i].sel(lat=slice(26,28), lon=slice(267,274))
        surf = surf.sel(time=is_jj(surf['time.month'])).mean(dim = ('lon','lat'))
        surf.name = ['surf_' + i]
        ds_surf2['surf_' + i] = surf



In [9]:
ds_surf = xr.auto_combine([ds_surf1, ds_surf2])

In [10]:
ds_surf

<xarray.Dataset>
Dimensions:      (time: 172)
Coordinates:
  * time         (time) object 2015-06-15 00:00:00 ... 2100-07-15 12:00:00
Data variables:
    surf_thetao  (time) float32 28.837887 30.826143 ... 32.18844 33.617107
    surf_so      (time) float32 36.4151 36.327785 ... 38.279163 38.324615
    surf_zos     (time) float32 -0.24073197 -0.1882354 ... -0.14569959